In [18]:
import numpy as np

# Функция для расчёта стоимости решения 
# (сумма всех проходимых по маршруту расстояний)
def cost(T, D, n):
    
    L = 0
    for i in range(n):
        if i != n - 1:
            L += D[T[i]][T[i + 1]]
        else:
            L += D[T[i]][T[0]]
    return L

# Функция для изначальной расстановки муравьёв
def make_ants(N, n):
    
#     Строка в матрице ant_matrix – муравей, столбец – вершина
#     ij-ое значение представляет собой маркер присутствия
#     i-го муравья в j-ой вершине.
#     Значения: 1 – находится в вершине, 0 – не был в вершине, -1 – был в вершине
#     Например, если значение [2][3] составляет 1, это означает, что
#     2-ой муравей сейчас находится в 3 вершине

    ant_matrix = np.zeros((N, n))
    i = 0
    j = n
#     K – показывает, во сколько раз кол-во муравьёв больше кол-ва вершин
    K = int(N / n)
    for k in range(K):
#         Помещаем i-го муравья в j-ый город, так чтобы i = j
#         Т.е., на главную диагональ ant_matrix ставим 1
        ant_matrix[i: j] = np.eye(n)
        i += n
        j += n
    return ant_matrix

# Функция для поика следующей вершины
def peak_search(ant, a, b, tau, D, i):
    
#     ant – текущий муравей
#     a, b – коэффициенты
#     tau – матрица феромонов
#     D – матрица расстояний между вершинами
#     i – текущая вершина

#     P_max – наибольшая вероятность перехода в j-ую вершину
    P_max = 0
#     Проходимся по всем вершинам, в которые может пойти муравей
    for j, value in enumerate(ant):
        if value == 0:
            if np.random.randint(0, 2) == 1:
                P = (tau[i][j]**a)/(D[i][j]**b) + np.random.randint(0, 10)
            else:    
                P = (tau[i][j] ** a) / (D[i][j] ** b)
            if P > P_max:
                P_max = P.copy()
                j_max = j
    return j_max

# Функция для расчёта феромонов, оставленных одним муравьём на своём пути
def pheromones(F, Q, L, T, n):
    for i in range(n):
        if i != n - 1:
            F[T[i]][T[i + 1]] += Q / L
        else:
            F[T[i]][T[0]] += Q / L
    return F

def best_path(D):
    
#     Константные параметры, которые требуется подобрать методом грубой силы
    A = [0.8, 1, 1.8]
    B = [0.6, 0.8]
#     n – количество вершин
    n = len(D)
#     T_best – массив для сохранения маршрутов
#     Зададим начальное решение
    T_best = np.arange(n)
#     Стоимость начального решения
    L_best = cost(T_best, D, n)
#     Перебираем комбинации параметров a и b
#     (элементов матриц A и B соответственно)
    for b in B:
        for a in A:
#             Задаём параметры алгоритма
#             Порядок цены оптимального решения Q
            Q = 100
#             Интенсивность испарения p
            p = 0
#             Количество муравьёв N
#             (которое должно быть кратно количеству вершин)
            N = 1 * n
#             Количество эпох epochs
            epochs = list(range(101))
#             Задаём начальную концентрацию феромонов в виде матрицы tau
            tau = np.random.uniform(1, 2, size=(n, n))
            tau[np.diag_indices_from(tau)] = 0
#             Цикл по эпохам (одна эпоха – все муравьи прошли путь)
            for epoch in epochs:
#                 Создаём муравьёв
                ant_matrix = make_ants(N, n)
#                 Формируем матрицу решений (маршрутов)
#                 i-ая строка – маршрут i-го муравья
                T = np.zeros((N, n), dtype=int)
#                 Формируем матрицу стоимостей решений
#                 i-ое значение – стоимость пути i-го муравья
                L = np.zeros(N, dtype=int)
#                 Формируем матрицу оставляемых феромонов
#                 ij-ый элемент – феромон на пути от i-ой к j-ой вершине 
                F = np.zeros((n, n))
#                 Проходим по всем муравьям
                for ant_index, ant in enumerate(ant_matrix):
#                     Фиксируем начальное положение муравья
                    T[ant_index][0] = np.where(ant == 1)[0][0]
#                     Проходимся по всем вершинам,
#                     чтобы отыскать ту, в которую муравей отправится дальше
                    for next_peak in range(1, n):
#                         i – текущее местоположение муравья
                        i = np.where(ant == 1)[0][0]
#                         j – следующая вершина
                        j = peak_search(ant, a, b, tau, D, i)
                        ant_matrix[ant_index][i] = -1
                        ant_matrix[ant_index][j] = 1
                        T[ant_index][next_peak] = j
#                     Стоимость пути текущего муравья
                    L[ant_index] = cost(T[ant_index], D, n)
#                     Формируем феромоны, оставляемые текущим муравьём
                    F = pheromones(F, Q, L[ant_index], T[ant_index], n)
#                 Обновляем уровни феромонов
                tau = tau * (1 - p) + F
#                 Выбираем наилучшее решение
                if L.min() < L_best:
                    L_best = L.min()
                    T_best = T[L.argmin()]
                
    return L_best

# Формируем матрицу D, в которой ij-ый элемент представляет собой 
# проходимое расстояние при переходе из i-ой вершины в j-ую.

# Вводим матрицу построчно 
# Сначала отдельно считываем первую строку, чтобы понять, сколько вершин
first_row = list(map(int, input().split()))
# Сохраняем количество вершин n
n = len(first_row)
D = np.zeros((n, n))
D[0] += first_row
for i in range(1, n):
    a = list(map(int, input().split()))
    D[i] += a

print(best_path(D))